# CLIMADA DEMO

Prepared by G. Aznar Siguan

The functionality of climada is gathered in the following classes:
* Entity: socio-economic models/data
  * Exposures: exposed values
  * ImpactFuncSet: collection of impact functions per hazard
  * DiscRates: discount rates
  * MeasureSet: collection of measures for adaptation
* Hazard: meteorological models/data
  * TropCyclone: tropical cyclone hazards
* Impact: impact results

Currently climada is able to read and load data of files containing the already modelled data (as for instance products of the climada MATLAB code), to visualize those and perform some basic analysis, and to compute the impact. Climada in Python can also model tropical cyclones from the IBTrACS tracks repository and generate probabilistic ones.

After the installation of climada, and with the climada_env virtual environment activated (if using virutal environments), some of climada classes, functions and constants are directly ready to import. For developers: they are the ones which are defined inside the \__all\__ command on top of each climada module.

# Case: Hurricane Andrew (Florida 1992)

## Entity

The entity class is just a container for the exposures, impact functions, discount reates and measures. It can be set with default values just by instantiation.
 
The data used as default is configurable through a configuration file. Climada comes with a configuration file and the user is free to define his own file by following the instructions in [Configuration options](https://github.com/davidnbresch/climada_python/blob/master/README.md).

In [ ]:
from climada import Entity
ent_fl = Entity() # Entity class instantiation with default values

The `impact_funcs` attribute is of type `ImpactFuncSet`. As such, it contains different impact functions for different hazards:

In [ ]:
% matplotlib inline
print("List of impact function's ids for every hazard type:")
print(ent_fl.impact_funcs.get_ids()) # List of impact function's ids for every hazard type
print('\nImpact functions graphs:')
ent_fl.impact_funcs.plot('TC'); # tropical cyclone impact functions
ent_fl.impact_funcs.plot('EQ'); # earthquake impact functions

Information and exercises about impact functions, numpy and matplotlib in [Hands-on_II](Hands-on_II.ipynb#climada-I:-Impact-Functions-(using-numpy and-matplotlib).

The `measures` attribute is of type `MeasureSet`. This class is a container of `Measure` instances, the same as `ImpactFuncSet` containing several `ImpactFunc`. This class is not used for impact calculations, but for building adaptation strategies:

In [ ]:
help(ent_fl.measures)

The `disc_rates` attribute is of type `DiscRates` and it simply contains the discount rates for every year, currently:

In [ ]:
ent_fl.disc_rates.plot()

Finally, the `exposures` attribute contains geolocalized values of anything exposed to the hazard, let it be monetary value of assets or number of human lifes, for example. It is of type `Exposures`. 

In this case, we want to fix the exposures for the assets located in Florida. We will use the data in file `ENT_FL_MAT`.

In [ ]:
from climada import Exposures, ENT_FL_MAT

ent_fl.exposures = Exposures(ENT_FL_MAT)
ent_fl.exposures.value_unit = 'USD'      # fill missing ('NA') value unit 
ent_fl.exposures.plot(ignore_null=True); # plot removing 0 values

The detailed description and exercises on the `Exposures` class can be found in [Hands-on_III - Exposures](Hands-on_III.ipynb#Exposures-class). There is a brief explanation of the libraries pandas and cartopy as well.

## Hazard tropical cyclone

### Centroids

The centroids are the geographical coordinates where there is information about the hazard. Usually, the hazard events are interpolated to these coordinates. They can be defined through the class `Centroids`.

In this example, we will a grid that covers Florida and the hurricane Andrew. This can be something like that:

In [ ]:
import numpy as np
from climada import Centroids
cent_fl = Centroids()
cent_fl.coord = (np.mgrid[22 : 35 : complex(0, 100), -94 : -71 : complex(0, 100)]).reshape(2, 10000).transpose()
cent_fl.id = np.arange(cent_fl.lat.size)
cent_fl.plot();

See [Hands-on_III - Centroids](Hands-on_III.ipynb#Centroids-class).

### TropCyclone

Tropical cyclones can be modelled from historical event tracks, in particular from [IBTrACS](https://www.ncdc.noaa.gov/ibtracs/) data. `TC_ANDREW_FL` file contains the IBrTRACS track of hurricane Andrew in 1992 over Florida.

In [ ]:
from climada import TropCyclone, TC_ANDREW_FL

In [ ]:
tc_andrew = TropCyclone() # instance TropCyclone
tc_andrew.set_from_tracks(TC_ANDREW_FL, centroids=cent_fl) # set hazard from track file with given centroids
tc_andrew.plot_tracks()                            # plot track
tc_andrew.plot_intensity(tc_andrew.event_name[0]); # plot intensity of the hazard

Synthetic events from the historical events can be generated in a `TropCyclone`:

In [ ]:
tc_andrew.set_random_walk(centroids=cent_fl)
print('Number of tracks:', len(tc_andrew.tracks))
tc_andrew.plot_tracks() # Blue lines represent historical events, black synthetic

A more completed set of tropical cyclones events in Florida can be found in file `HAZ_DEMO_MAT`. This contains 1445 historical events from year 1851 to 2011 and 9 synthetical events for each historical one.

In [ ]:
from climada import HAZ_DEMO_MAT
tc_fl = TropCyclone(HAZ_DEMO_MAT, 'Historic and synthetic tropical cyclones in Florida from 1851 to 2011.')

There are two `ANDREW` events in Florida:

In [ ]:
print('The first ANDREW happened in', tc_fl.get_date_strings('ANDREW')[0])
print('The second ANDREW happened in', tc_fl.get_date_strings('ANDREW')[1])
tc_fl.plot_intensity(event='ANDREW'); # Plot all ANDREW events

In a specific point, for example (lat, lon) = (26, -81), the tropical cyclones have had the following maximum intensities:

In [ ]:
tc_fl.plot_intensity(centr=(26, -81));

Also, the intensity maps for different return periods can be representative for such an amount of events:

In [ ]:
tc_fl.plot_stats();

To see how the `TropCyclone` class works, see [Hands-on_IV](Hands-on_IV.ipynb#climada-III:-Hazard-and-TropicCyclone-(using-xarray-and-pint), where the `Hazard` class and `trop_cyclone` module are explained, as well as the xarray and pint libraries.

## Impact

The impact of hazard events over an entity can be computed easily from the previously explained classes. The `Impact` class computes this information.

This is the impact from ANDREW tropical cyclone and its probabilistic events:

In [ ]:
from climada import Impact
imp_andrew = Impact()
# Compute impact of hurricane Andrew in tc_andrew over Florida entity:
imp_andrew.calc(ent_fl.exposures, ent_fl.impact_funcs, tc_andrew)
imp_andrew.plot_at_exposure(); # accumulated losses for each exposure

This is the impact exceedence frequency curve of all the historical and synthetic events over Florida:

In [ ]:
# remove previous assigned centroids
ent_fl.exposures.assigned = dict()
# compute new impact
imp_fl = Impact()
# Compute impact over Florida:
imp_fl.calc(ent_fl.exposures, ent_fl.impact_funcs, tc_fl)
freq_curve_fl = imp_fl.calc_freq_curve() # impact exceedence frequency curve
freq_curve_fl.plot();

Finally, we can save our variables in pickle format using the `save` function. This will save your results in the folder specified in the configuration file. The default folder is a `results` folder which is created in the current path.

In [ ]:
from climada import save
save('impact_florida.pkl', imp_fl)

Later, the data can be read as follows:

In [ ]:
import pickle
import os
abs_path = os.path.join(os.getcwd(), 'results/impact_florida.pkl') # absolute path
with open(abs_path, 'rb') as f:
    data = pickle.load(f)

print('Data read:', type(data))

# Your case

### EXERCISE:

1. Build an entity. It might be one from your previous runs in MATLAB. Make sure i'is saved in version > v7.3 if it's a MATLAB file. If it's not, you'll get an error message. Then, you can save it again in MATLAB like that:
`save('file_name.mat','variable_name', '-v7.3')`

2. Build a hazard. It might also come from a previous run in MATLAB. This file might already contain the centroids. If not, define the centroids as well and use them in your calculations.

3. Compute the impact.

4. Visualization. Plot:
   * the damage functions for the hazard
   * the entity values map
   * the strongest event intensity
   * the maximum hazard intensity of all the events in Zürich (47.38, 8.54)
   * the impact exceedence frequency curve


In [ ]:
# Put your code here






In [ ]:
# SOLUTION: example: winter storms in europe
# Put any absoulte path for your files or set up the configuration variable "repository"
from climada import DATA_DIR
FILE_HAZARD = DATA_DIR + '/demo/WS_ERA40.mat' 
FILE_ENTITY = DATA_DIR + '/demo/WS_Europe.xls'

from climada import Hazard, Exposures, ImpactFuncSet, Impact

# 1. Entity: we only need impact functions and exposures to compute the impact
# Exposures
exp_ws_eu = Exposures()
# as seen in Hands-on_III, the variable names in the file can be changed
var_names = exp_ws_eu.get_def_file_var_names('.xls')
var_names['col_name']['imp'] = 'VulnCurveID'
exp_ws_eu.read(FILE_ENTITY, 'EU entity for winter storms.', var_names=var_names)

# Impact functions
impf_ws_eu = ImpactFuncSet()
var_names = impf_ws_eu.get_def_file_var_names('.xls')
var_names['col_name']['func_id'] = 'VulnCurveID'
impf_ws_eu.read(FILE_ENTITY, 'Impact functions for winter storms in EU.', var_names=var_names)

# 2. Hazard
haz_ws_eu = Hazard('WS', FILE_HAZARD, 'WS EU ERA 40')

# 3. Impact
imp_ws_eu = Impact()
imp_ws_eu.calc(exp_ws_eu, impf_ws_eu, haz_ws_eu)

# 4.
# the damage functions for the hazard
impf_ws_eu.plot()

# the entity values map
exp_ws_eu.plot(pop_name=False)

# the strongest event
haz_ws_eu.plot_intensity(-1) # might be better to use an other earth projection?

# the hazard intensity maps for different return periods
haz_ws_eu.plot_intensity(centr=(47.38, 8.54))

# the impact exceedence frequency curve
imp_exc_curve = imp_ws_eu.calc_freq_curve()
imp_exc_curve.plot()